In [183]:
import math
N = 0
START = 0.1
STOP = 1
EPS = 0.01
def function(x):
  global N
  N += 1
  return 10*x*math.log(x) - (x**2 / 2)

In [184]:
import numpy as np
import pandas as pd

In [185]:
df = pd.DataFrame(columns=["name","minArg", "minFunction", "N"])

In [186]:
#Перебор
def perebor(start, stop, epsilon):
  numberOfSteps = int((stop-start)/epsilon)
  minF = function(start)
  minArg = start
  for val in np.linspace(start, stop, num=numberOfSteps):
    eval = function(val)
    if (minF > eval):
      minF = eval
      minArg = val
  return (minF, minArg)

In [187]:
(minF, minArg) = perebor(START, STOP, EPS)
print(minF, minArg, N)
df = pd.concat([df, pd.Series({"name": 'Перебор',"minArg": minArg, "minFunction": minF, "N": N}).to_frame().T], ignore_index=True)
N = 0

-3.749070050397621 0.3831460674157303 91


In [188]:
# Поразрядный поиск
def porPoisk(start, stop, epsilon):
  step = (stop - start)/4
  if step < epsilon:
    print(f'step > epsilon, step={step} epsilon={epsilon}')
  minArg = val = start
  fNext = function(minArg)
  minF = 0
  while abs(step) > epsilon:
    val += step
    f = fNext
    fNext = function(val)

    if fNext > f:
      minF = f;
      minArg = val
      step /= -4
      print('change direction')
  return (minF, minArg)
  

In [189]:
(minF, minArg) = porPoisk(START, STOP, EPS)
print(minF, minArg, N)
df = pd.concat([df, pd.Series({"name": 'Поразрядный поиск',"minArg": minArg, "minFunction": minF, "N": N}).to_frame().T], ignore_index=True)
N = 0

change direction
change direction
change direction
-3.749069344668216 0.39531249999999984 12


In [190]:
# Дихотомия
def dihotomia(start, stop, epsilon, delta):
  if (0 > delta) or (delta > 2 * epsilon):
    print('invalid q')
    return 0
  leftPoint = (start + stop - delta) / 2
  rightPoint = (start + stop + delta) / 2
  while (stop - start)/2 > epsilon:

    if function(leftPoint) < function(rightPoint):
      stop = rightPoint
    else:
      start = leftPoint
    leftPoint = (start + stop - delta) / 2
    rightPoint = (start + stop + delta) / 2
  minArg = (start + stop) / 2
  minF = function(minArg)
  return (minF, minArg)

In [191]:
(minF, minArg) = dihotomia(START, STOP, EPS, EPS)
print(minF, minArg, N)
df = pd.concat([df, pd.Series({"name": 'Дихотомия',"minArg": minArg, "minFunction": minF, "N": N}).to_frame().T], ignore_index=True)
N = 0

-3.748998103482821 0.37964843750000005 15


In [192]:
# Золотое сечение
from math import sqrt


def golden(start, stop, epsilon):
  tau = (sqrt(5) - 1) / 2
  leftPoint = start + (1 - tau)*(stop - start)
  rightPoint = start + tau*(stop - start)
  leftF = function(leftPoint)
  rightF = function(rightPoint)
  while abs(stop - start) > epsilon:
    if leftF >= rightF:
      start = leftPoint
      leftPoint = rightPoint
      leftF = rightF
      rightPoint = start + tau*(stop - start)
      rightF = function(rightPoint)
    else:
      stop = rightPoint
      rightPoint = leftPoint
      rightF = leftF
      leftPoint = start + (1 - tau)*(stop - start)
      leftF = function(leftPoint)
  minArg = (start + stop) / 2
  minF = function(minArg)
  return (minF, minArg)
    

In [193]:
(minF, minArg) = golden(START, STOP, EPS)
print(minF, minArg, N)
df = pd.concat([df, pd.Series({"name": 'Золотое сечение',"minArg": minArg, "minFunction": minF, "N": N}).to_frame().T], ignore_index=True)
N = 0

-3.7490787329225768 0.38263779380403007 13


In [194]:
def parabola(start, stop, epsilon):
  (x1, x2, x3, f1, f2, f3) = goldPoints(start, stop)

  a1 = (f2 - f1)/(x2 - x1)
  a2 = 1/(x3 - x2) * ((f3 - f1)/(x3 - x1) - (f2 - f1)/(x2 - x1))
  newX = (x1 + x2 - a1/a2)/2

  if (x1<newX<x2<x3):
    x1 = newX
    f1 = function(newX)
  
  if ((x1<x2<newX<x3)):
    x3 = newX
    f3 = function(newX)

  oldX = newX

  a1 = (f2 - f1)/(x2 - x1)
  a2 = 1/(x3 - x2) * ((f3 - f1)/(x3 - x1) - (f2 - f1)/(x2 - x1))
  newX = (x1 + x2 - a1/a2)/2

  while abs(oldX - newX) < epsilon:

    if (x1<newX<x2<x3):
      x1 = newX
      f1 = function(newX)
  
    if ((x1<x2<newX<x3)):
      x3 = newX
      f3 = function(newX)

    oldX = newX

    a1 = (f2 - f1)/(x2 - x1)
    a2 = 1/(x3 - x2) * ((f3 - f1)/(x3 - x1) - (f2 - f1)/(x2 - x1))
    newX = (x1 + x2 - a1/a2)/2
  
  minArg = newX
  minF = function(newX)
  return (minF, minArg)

def goldPoints(start, stop):
  tau = (sqrt(5) - 1) / 2
  leftPoint = start + (1 - tau)*(stop - start)
  rightPoint = start + tau*(stop - start)
  leftF = function(leftPoint)
  rightF = function(rightPoint)
  startF = function(start)
  stopF = function(stop)
  while True:

    if (startF >= leftF < stopF) or (startF > leftF <= stopF):
      return (start, leftPoint, stop, startF, leftF, stopF)

    if (leftF >= rightF < stopF) or (leftF > rightF <= stopF):
      return (leftPoint, rightPoint, stop, leftF, rightF, stopF)

    if leftF >= rightF:
      start = leftPoint
      leftPoint = rightPoint
      startF = leftF
      leftF = rightF
      rightPoint = start + tau*(stop - start)
      rightF = function(rightPoint)
    else:
      stop = rightPoint
      rightPoint = leftPoint
      stopF = rightF
      rightF = leftF
      leftPoint = start + (1 - tau)*(stop - start)
      leftF = function(leftPoint)

In [195]:
(minF, minArg) = parabola(START, STOP, EPS)
print(minF, minArg, N)
df = pd.concat([df, pd.Series({"name": 'Парабола',"minArg": minArg, "minFunction": minF, "N": N}).to_frame().T], ignore_index=True)
N = 0

-3.7448613264581905 0.4006774247987953 6


In [196]:
df

,name,minArg,minFunction,N
0,Перебор,0.383146,-3.74907,91
1,Поразрядный поиск,0.395312,-3.749069,12
2,Дихотомия,0.379648,-3.748998,15
3,Золотое сечение,0.382638,-3.749079,13
4,Парабола,0.400677,-3.744861,6
